### QA 
1. input molecules graph data √
2. input correct env variable √  - btw: art is what seawater?
3. connect these data by Useing environmental variables as Transformer Positional Encoding

### 1. Load molecules graph datas from pickle

In [4]:
## Load the molecular data (.pkl) for molecules with Targeted BDE & BDFE 

import os
import pickle
import networkx as nx
from torch_geometric.utils import from_networkx

folder_path = r"C:\Users\80710\OneDrive - Imperial College London\2025 engineering\GNN molecules\graph_pickles"

# Get all .pkl file paths in the folder
pkl_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".pkl")]

# Load all graphs
graph_list = []
for file_path in pkl_files:
    with open(file_path, "rb") as file:
        graph = pickle.load(file)  # Load each .pkl graph
        graph_list.append(graph)  # Store in a list

print(f"Loaded {len(graph_list)} graphs")


Loaded 20 graphs


In [5]:
## 检查
import pandas as pd

# Select the first graph for visualization
nx_graph = graph_list[0]

# Convert node data to DataFrame
node_data = pd.DataFrame.from_dict(dict(nx_graph.nodes(data=True)), orient="index")

# Convert edge data to DataFrame
edge_data = pd.DataFrame.from_records([(u, v, d) for u, v, d in nx_graph.edges(data=True)], 
                                      columns=["source", "target", "attributes"])

# Expand edge attributes
if not edge_data.empty and isinstance(edge_data.iloc[0]["attributes"], dict):
    attr_df = edge_data["attributes"].apply(pd.Series)  # Convert dict to columns
    edge_data = pd.concat([edge_data.drop(columns=["attributes"]), attr_df], axis=1)

# Display DataFrames
from IPython.display import display
display(node_data)
display(edge_data)



,symbol,rdkit_idx
0,C,0.0
1,C,1.0
2,C,2.0
3,C,3.0
4,C,4.0
5,C,5.0
6,C,6.0
7,C,7.0
8,C,8.0
9,C,9.0


,source,target,bond_index,bde_pred,bdfe_pred
0,0,1,0.0,89.382645,75.711845
1,0,H_23,25.0,100.077171,91.049126
2,1,2,1.0,85.872452,71.412849
3,1,H_27,29.0,97.163109,87.689636
4,2,3,2.0,85.041306,70.000282
5,2,H_28,30.0,95.392189,86.257263
6,3,4,3.0,83.115479,66.995277
7,3,H_30,32.0,94.518471,84.748619
8,4,5,NaN,NaN,NaN
9,4,10,NaN,NaN,NaN


Thinking: 为什么rdix里H对应的都是NAN 我们转化时用的是H-H 单H 键不需要 再进入rdix编号

备注：os.listdir... 列出文件夹中文件名称，迭代获取，保留.pkl,output list
     "rb"对口pkl二进制

In [ ]:
## Convert Graphs to PyTorch Geometric Format for GNN model
# 方便训练和GNN计算，转化为PyG

import os
import pickle
import torch
import pandas as pd
from torch_geometric.data import Data
import networkx as nx

# Path to dataset folder (Update this)
folder_path = r"C:\Users\80710\OneDrive - Imperial College London\2025 engineering\GNN molecules\graph_pickles"

# Get all .pkl file paths
pkl_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".pkl")]

# List to store PyTorch Geometric graphs
pytorch_graphs = []

# **Function to handle missing hydrogen atom labels**
def clean_node_labels(node_data):
    """Replace NaN hydrogen labels with unique identifiers."""
    return {node: idx if not pd.isna(node) else f"H_{idx}" for idx, node in enumerate(node_data)}

# 分别提取节点和边的数据重构graph
# **Loop through each .pkl file and convert to PyG format**
for i, file_path in enumerate(pkl_files):
    with open(file_path, "rb") as file:
        nx_graph = pickle.load(file)  # Load NetworkX graph

        if isinstance(nx_graph, nx.Graph):
            # **Relabel nodes to avoid NaN values**
            mapping = clean_node_labels(nx_graph.nodes())
            nx_graph = nx.relabel_nodes(nx_graph, mapping)

            # **Convert Edges to PyG format (edge_index)**
            edge_index = torch.tensor(list(nx_graph.edges()), dtype=torch.long).T
            if edge_index.numel() == 0:  # Handle cases with no edges
                edge_index = torch.empty((2, 0), dtype=torch.long)

            # **Extract Node Features (Use Atomic Symbols)**
            first_node = next(iter(nx_graph.nodes), None)
            if first_node is not None and "symbol" in nx_graph.nodes[first_node]:
                node_features = torch.tensor(
                    [[1 if nx_graph.nodes[n]["symbol"] == "C" else 0] for n in nx_graph.nodes()],
                    dtype=torch.float
                )  # One-hot encoding Carbon (C) vs Hydrogen (H)
            else:
                node_features = torch.zeros((len(nx_graph.nodes()), 1), dtype=torch.float)  # Default: 1D zero feature

            # **Extract Edge Features (BDE, BDFE) - Handle NoneType**
            edge_features_list = []
            for u, v in nx_graph.edges():
                bde = nx_graph[u][v].get("bde_pred", 0.0)  # Replace None with 0.0
                bdfe = nx_graph[u][v].get("bdfe_pred", 0.0)  # Replace None with 0.0
                edge_features_list.append([bde if bde is not None else 0.0, bdfe if bdfe is not None else 0.0])

            edge_features = torch.tensor(edge_features_list, dtype=torch.float) if edge_features_list else None

            # **Create PyG Graph**
            pyg_graph = Data(x=node_features, edge_index=edge_index, edge_attr=edge_features)

            # **Store Processed Graph**
            pytorch_graphs.append(pyg_graph)
            print(f"Converted Graph {i+1}: {len(nx_graph.nodes)} nodes, {len(nx_graph.edges)} edges")

print(f"\n Total converted graphs: {len(pytorch_graphs)}")


Converted Graph 1: 42 nodes, 44 edges
Converted Graph 2: 43 nodes, 45 edges
Converted Graph 3: 52 nodes, 56 edges
Converted Graph 4: 54 nodes, 58 edges
Converted Graph 5: 56 nodes, 60 edges
Converted Graph 6: 56 nodes, 60 edges
Converted Graph 7: 58 nodes, 62 edges
Converted Graph 8: 58 nodes, 62 edges
Converted Graph 9: 60 nodes, 64 edges
Converted Graph 10: 60 nodes, 64 edges
Converted Graph 11: 62 nodes, 66 edges
Converted Graph 12: 62 nodes, 66 edges
Converted Graph 13: 46 nodes, 48 edges
Converted Graph 14: 48 nodes, 50 edges
Converted Graph 15: 52 nodes, 54 edges
Converted Graph 16: 52 nodes, 54 edges
Converted Graph 17: 53 nodes, 55 edges
Converted Graph 18: 53 nodes, 55 edges
Converted Graph 19: 48 nodes, 52 edges
Converted Graph 20: 48 nodes, 52 edges

 Total converted graphs: 20


In [61]:
print(pytorch_graphs[0])


Data(x=[42, 1], edge_index=[2, 44], edge_attr=[44, 2])


---

## 2. Load the environmental variable

In [ ]:
import pandas as pd

# Load environmental data from Excel
env_data_path = r"C:\Users\80710\OneDrive - Imperial College London\2025 engineering\GNN molecules\graph_pickles\dataset02.xlsx"
df_env = pd.read_excel(env_data_path, engine='openpyxl')

# Columns of interest
env_columns = ["temperature", "seawater", "time", "concentration"]

# Convert 'seawater' column properly
df_env["seawater"] = df_env["seawater"].map({"sea": 0, "art": 1}).astype(float)

# Convert all columns to numeric (force object types to float)
df_env = df_env.apply(pd.to_numeric, errors='coerce')

df_env = df_env.drop(columns=['component'])


# Fill missing values (optional, based on needs)
df_env = df_env.dropna()


# Print column types again to verify
print(df_env.dtypes)
print(df_env.isna().sum())  # Count NaNs in each column

print(df_env.head())




data number         float64
temperature         float64
seawater            float64
concentration         int64
time                  int64
BDE                 float64
BDFE                float64
energy              float64
degradation_rate    float64
dtype: object
data number         0
temperature         0
seawater            0
concentration       0
time                0
BDE                 0
BDFE                0
energy              0
degradation_rate    0
dtype: int64
   data number  temperature  seawater  concentration  time        BDE  \
0          1.0         35.6       0.0             70    30  91.907407   
1          1.0         35.6       0.0             70    30  91.700000   
2          1.0         35.6       0.0             70    30  91.674194   
3          1.0         35.6       0.0             70    30  91.657576   
4          1.0         35.6       0.0             70    30  91.778378   

        BDFE       energy  degradation_rate  
0  80.962963  -975.632487          0.6

---

## 3. Use environmental variables as Transformer Positional Encoding connect

simply use nx graph is enough

In [78]:
from torch_geometric.data import Data

class MoleculeEnvDataset(torch.utils.data.Dataset):
    def __init__(self, graphs, df_env):
        self.graphs = graphs  # List of PyG Data objects
        self.df_env = df_env  # Environmental features & target values (DataFrame)

    def __getitem__(self, idx):
        graph = self.graphs[idx]  # PyG Data object

        # Convert environmental features to tensor
        env_features = torch.tensor(self.df_env.iloc[idx, :-1].values, dtype=torch.float32)  
        target = torch.tensor(self.df_env.iloc[idx, -1], dtype=torch.float32)  # Last column is target

        # **Store them inside the graph object**
        graph.env_features = env_features.unsqueeze(0)  # Ensure correct shape
        graph.targets = target.unsqueeze(0)  

        return graph

    def __len__(self):
        return len(self.graphs)


In [70]:
from torch.utils.data import Dataset, DataLoader

class MoleculeEnvDataset(Dataset):
    def __init__(self, graphs, df_env):
        self.graphs = graphs
        self.df_env = df_env.reset_index(drop=True)
    
    def __len__(self):
        return len(self.graphs)
    
    def __getitem__(self, idx):
        graph = self.graphs[idx]  # PyG graph

        # Extract environment features
        env_values = df_env.iloc[idx, [1, 2, 3, 4]].values  # Extract temperature, seawater, concentration, time
        print(f"Index {idx} - Env Values:", env_values)  # Debugging print

        # Convert to tensor (use .astype(float) to prevent PyTorch issues)
        env_features = torch.tensor(env_values.astype(float), dtype=torch.float32)

        # Extract and convert target value
        target_value = df_env.iloc[idx, -1]
        print(f"Index {idx} - Target Value:", target_value)  # Debugging print

        target = torch.tensor(float(target_value), dtype=torch.float32)  # Ensure it's a single float

        return graph, env_features, target



### 3.1  建立Build DataLoader
PYG的Data对象代表图形，图形在节点和边缘的数量上有所不同。默认的DataLoader试图像张量一样堆叠它们，是不行滴。
 Custom Collate？

In [71]:
from torch_geometric.data import Batch

def my_collate(batch):
    graphs, envs, targets = zip(*batch)
    graphs = Batch.from_data_list(graphs)  # Batch PyG graphs
    envs = torch.stack(envs)  # Convert list of tensors to a batch tensor
    targets = torch.stack(targets)  # Convert list of targets to tensor

    return graphs, envs, targets

# Create Dataset
dataset = MoleculeEnvDataset(graph_list, df_env)

# Create DataLoader
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=my_collate)


### 3.2 建立 env + molecules graph层 → 建立GNN
由于PYG将多个图批量分为一个大图结构，因此我们需要data.batch来确保每个节点都会收到其相应图的正确环境嵌入
1.PYG将多个图形组合在一起→来自不同图形的节点被连接
2.data.batch tracks nodes属于哪个图。
3.我们使用env_pos_emb[data.batch]添加每个节点的正确环境效果
4.这确保每个节点都会获取其相应的环境编码

In [72]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class EnvPositionalEncoder(nn.Module):
    """ Encodes environmental variables into a learnable embedding. """
    def __init__(self, env_input_dim, d_model, hidden_dim=64):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(env_input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, d_model)
        )

    def forward(self, env_features):
        return self.mlp(env_features)

class SimpleGraphModel(nn.Module):
    def __init__(self, num_node_features, env_input_dim, hidden_dim=128, output_dim=1):
        super(SimpleGraphModel, self).__init__()
        
        # Graph neural network layers
        self.conv1 = GCNConv(num_node_features, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)

        # Environment encoding
        self.env_encoder = EnvPositionalEncoder(env_input_dim, hidden_dim)

        # Final MLP for regression
        self.mlp = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, data, env_features):
        x, edge_index = data.x, data.edge_index
        
        # Graph embeddings
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        graph_embedding = x.mean(dim=0)  # Global pooling (mean over all nodes)

        # Environment embeddings
        env_embedding = self.env_encoder(env_features)

        # Concatenate both representations
        combined = torch.cat([graph_embedding, env_embedding], dim=-1)

        # Predict degradation rate
        return self.mlp(combined)


### 3.3 MSE/RMSE/R2 检验 → 测试model

In [73]:
import torch.optim as optim
import math
from sklearn.metrics import r2_score

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def train_model(model, dataloader, epochs=20):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.MSELoss()

    for epoch in range(1, epochs + 1):
        model.train()
        total_loss = 0.0
        total_samples = 0
        
        for graphs_batch, envs_batch, targets_batch in dataloader:
            graphs_batch = graphs_batch.to(device)
            envs_batch = envs_batch.to(device)
            targets_batch = targets_batch.to(device)

            optimizer.zero_grad()
            outputs = model(graphs_batch, envs_batch)

            # Compute MSE Loss
            loss = criterion(outputs, targets_batch)
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * targets_batch.size(0)
            total_samples += targets_batch.size(0)

        avg_loss = total_loss / total_samples  # Compute average MSE loss

        # === Evaluation Step ===
        model.eval()
        with torch.no_grad():
            preds, gts = [], []
            
            for graphs_batch, envs_batch, targets_batch in dataloader:
                graphs_batch = graphs_batch.to(device)
                envs_batch = envs_batch.to(device)
                targets_batch = targets_batch.to(device)

                out = model(graphs_batch, envs_batch)
                preds.append(out.cpu())
                gts.append(targets_batch.cpu())

            preds = torch.cat(preds).numpy()
            gts = torch.cat(gts).numpy()

            mse_val = F.mse_loss(torch.tensor(preds), torch.tensor(gts)).item()
            rmse_val = math.sqrt(mse_val)
            r2_val = r2_score(gts, preds)

        print(f"Epoch {epoch}/{epochs} - Train MSE: {avg_loss:.4f}, Val RMSE: {rmse_val:.4f}, R²: {r2_val:.4f}")




Using device: cpu


In [74]:
from torch_geometric.loader import DataLoader
dataloader = DataLoader(pytorch_graphs, batch_size=4, shuffle=True)


In [75]:
# Initialize and Train Model
num_node_features = 2  # BDE BDFE
env_input_dim = 4  # [temperature, seawater, time, concentration]
model = SimpleGraphModel(num_node_features, env_input_dim, hidden_dim=128, output_dim=1)

train_model(model, dataloader, epochs=20)

ValueError: too many values to unpack (expected 3)

---

In [79]:
train_model(model, dataloader, epochs=20)

ValueError: too many values to unpack (expected 3)

## 4. Training + fixed code

In [67]:
# Check dataset output format before dataloader
dataset = MoleculeEnvDataset(graphs, df_env)
print(dataset[0])  # Expecting (graph, env_features, target)


NameError: name 'graphs' is not defined

In [68]:
dataset = MoleculeEnvDataset(pytorch_graphs, df_env)
print(len(dataset))  # Should print the total number of graphs
print(dataset[0])    # Should print (graph, env_features, target)


20
Index 0 - Env Values: [35.6  0.  70.  30. ]
Index 0 - Target Value: 0.6709138594659749
(Data(x=[42, 1], edge_index=[2, 44], edge_attr=[44, 2]), tensor([35.6000,  0.0000, 70.0000, 30.0000]), tensor(0.6709))


In [76]:
for batch in dataloader:
    print(batch)
    break  # Stop after one batch


DataBatch(x=[220, 1], edge_index=[2, 236], edge_attr=[236, 2], batch=[220], ptr=[5])


In [77]:
for batch in dataloader:
    graphs_batch = batch  # Entire batch is a DataBatch object
    envs_batch = batch.env_features.to(device)  # Ensure this attribute exists
    targets_batch = batch.targets.to(device)    # Ensure this attribute exists


AttributeError: 'GlobalStorage' object has no attribute 'env_features'